# Notebook to motion magnify sets of videos

Uses learned motion mag: https://people.csail.mit.edu/tiam/deepmag/deepmag.pdf

In [5]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import sys

import argparse
import os
import shutil
import tensorflow as tf
import setproctitle
from configobj import ConfigObj
from validate import Validator

sys.path.append("../../deep_motion_mag/")

from magnet import MagNet3Frames

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


In [2]:
!ls ../../deep_motion_mag

configs				      preprocessor.py
convert_3frames_data_to_tfrecords.py  __pycache__
data				      README.md
data_loader.py			      requirements.txt
LICENSE				      run_on_test_videos.sh
magnet.py			      run_temporal_on_test_videos.sh
main.py				      train.sh
modules.py			      utils.py
ops.py


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
!nvidia-smi

Wed Jan 29 17:35:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:02:00.0 Off |                  N/A |
| 52%   81C    P2   220W / 250W |   8751MiB / 12196MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   33C    P8     9W / 250W |      0MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [17]:
# %%bash

# EXP_NAME="test"
# VIDEO=1003254
# AMPLIFICATION_FACTOR=10
# DYNAMIC_MODE="no"
# OUT_DIR=data/output/"$VIDEO"_"$EXP_NAME"_"$AMPLIFICATION_FACTOR"
# VID_DIR=data/vids/"$VIDEO"
# OUT_DIR=data/output/"$VIDEO"
# if [ ! -d "$OUT_DIR" ]; then
#     mkdir -p "$OUT_DIR"
# fi

# FLAGS="--phase=run --vid_dir=$VID_DIR --out_dir=$OUT_DIR --amplification_factor=$AMPLIFICATION_FACTOR"
# if [ "$DYNAMIC_MODE" = yes ] ; then
#     FLAGS="$FLAGS"" --velocity_mag"
# fi
# ../deep_motion_mag/main.py --config_file=configs/"$EXP_NAME".conf \
#     $FLAGS

In [18]:
# %%bash
# sh ../deep_motion_mag/run_on_test_videos.sh test1 1003254 10

In [19]:
def run_motion_mag(args):
    configspec = ConfigObj(args.config_spec, raise_errors=True)
    config = ConfigObj(args.config_file,
                       configspec=configspec,
                       raise_errors=True,
                       file_error=True)
    # Validate to get all the default values.
    config.validate(Validator())
    if not os.path.exists(config['exp_dir']):
        # checkpoint directory.
        os.makedirs(os.path.join(config['exp_dir'], 'checkpoint'))
        # Tensorboard logs directory.
        os.makedirs(os.path.join(config['exp_dir'], 'logs'))
        # default output directory for this experiment.
        os.makedirs(os.path.join(config['exp_dir'], 'sample'))
    network_type = config['architecture']['network_arch']
    exp_name = config['exp_name']
    setproctitle.setproctitle('{}_{}_{}' \
                              .format(args.phase, network_type, exp_name))
    tfconfig = tf.ConfigProto(allow_soft_placement=True,
                              log_device_placement=False)
    tfconfig.gpu_options.allow_growth = True

    with tf.Session(config=tfconfig) as sess:
        model = MagNet3Frames(sess, exp_name, config['architecture'])
        checkpoint = '../deep_motion_mag/'+config['training']['checkpoint_dir']
        if args.phase == 'train':
            train_config = config['training']
            if not os.path.exists('../deep_motion_mag/'+train_config['checkpoint_dir']):
                os.makedirs('../deep_motion_mag/'+train_config['checkpoint_dir'])
            model.train(train_config)
        elif args.phase == 'run':
            model.run(checkpoint,
                      args.vid_dir,
                      args.frame_ext,
                      args.out_dir,
                      args.amplification_factor,
                      args.velocity_mag)
        elif args.phase == 'run_temporal':
            model.run_temporal(checkpoint,
                               args.vid_dir,
                               args.frame_ext,
                               args.out_dir,
                               args.amplification_factor,
                               args.fl,
                               args.fh,
                               args.fs,
                               args.n_filter_tap,
                               args.filter_type)
        else:
            raise ValueError('Invalid phase argument. '
                             'Expected ["train", "run", "run_temporal"], '
                             'got ' + args.phase)


# Define parameters
exp_name= 'o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3'
phase = 'run'
config_file = '../deep_motion_mag/configs/'+exp_name+'.conf'
config_spec = '../deep_motion_mag/configs/configspec.conf'
vid_dir = '../../deepfake_data/fb_dfd_release_0.1_final/method_A_frames/1320815_1573558_A_002.mp4'
frame_ext = 'jpg'
out_dir = '../../deepfake_data/fb_dfd_release_0.1_final/method_A_frames_mm/1320815_1573558_A_002.mp4'
os.makedirs(out_dir, exist_ok=True)
amplification_factor = 5
velocity_mag = True
fl = 0.5
fh = 1
fs = 30
n_filter_tap = 2 
filter_type = 'DifferenceOfIIR'


class Parser:        
    def add_argument(self, name, dest='var', default=None, help='', required=True, type='', action=''):
        exec("self."+dest+" = default")
parser = Parser()
        
parser.add_argument('--phase', dest='phase', default=phase,
                    help='train, test, run, interactive')
parser.add_argument('--config_file', dest='config_file', required=True, default = config_file,
                    help='path to config file')
parser.add_argument('--config_spec', dest='config_spec', default=config_spec,
                    help='path to config spec file')

# for inference
parser.add_argument('--vid_dir', dest='vid_dir', default=vid_dir,
                    help='Video folder to run the network on.')
parser.add_argument('--frame_ext', dest='frame_ext', default=frame_ext,
                    help='Video frame file extension.')
parser.add_argument('--out_dir', dest='out_dir', default=out_dir,
                    help='Output folder of the video run.')
parser.add_argument('--amplification_factor', dest='amplification_factor',
                    type=float, default=amplification_factor,
                    help='Magnification factor for inference.')
parser.add_argument('--velocity_mag', dest='velocity_mag', action='store_true', default=velocity_mag,
                    help='Whether to do velocity magnification.')

# For temporal operation.
parser.add_argument('--fl', dest='fl', type=float, default=fl,
                    help='Low cutoff Frequency.')
parser.add_argument('--fh', dest='fh', type=float, default=fh,
                    help='High cutoff Frequency.')
parser.add_argument('--fs', dest='fs', type=float, default=fs,
                    help='Sampling rate.')
parser.add_argument('--n_filter_tap', dest='n_filter_tap', type=int, default=n_filter_tap,
                    help='Number of filter tap required.')
parser.add_argument('--filter_type', dest='filter_type', type=str, default=filter_type,
                    help='Type of filter to use, must be Butter or FIR.')

from pprint import pprint
pprint(vars(parser))

# run_motion_mag(parser)

{'amplification_factor': 5,
 'config_file': '../deep_motion_mag/configs/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3.conf',
 'config_spec': '../deep_motion_mag/configs/configspec.conf',
 'fh': 1,
 'filter_type': 'DifferenceOfIIR',
 'fl': 0.5,
 'frame_ext': 'jpg',
 'fs': 30,
 'n_filter_tap': 2,
 'out_dir': '../../deepfake_data/fb_dfd_release_0.1_final/method_A_frames_mm/1320815_1573558_A_002.mp4',
 'phase': 'run',
 'velocity_mag': True,
 'vid_dir': '../../deepfake_data/fb_dfd_release_0.1_final/method_A_frames/1320815_1573558_A_002.mp4'}


In [20]:
from tqdm import tqdm

def run_motion_mag_all_folders(base_dirs, config_spec, config_file):
    configspec = ConfigObj(config_spec, raise_errors=True)
    config = ConfigObj(config_file,
                       configspec=configspec,
                       raise_errors=True,
                       file_error=True)
    
    # Validate to get all the default values.
    config.validate(Validator())
    
    network_type = 'ynet_3frames'
    exp_name = '%(dataset)s_%(variant)s'
    setproctitle.setproctitle(exp_name)
    tfconfig = tf.ConfigProto(allow_soft_placement=True,
                              log_device_placement=False)
    tfconfig.gpu_options.allow_growth = True
    
    with tf.Session(config=tfconfig) as sess:
        model = MagNet3Frames(sess, exp_name, config['architecture'])
        checkpoint = '../deep_motion_mag/data/training/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3/checkpoint'
        for bd in base_dirs:
            print("Magnifying vids in", bd)
            for vid in tqdm(os.listdir(bd)):
                if not os.path.isdir(os.path.join(bd,vid)):
                    continue
                print('running motion mag for vid:',vid)
                input_folder = os.path.join(bd, vid)
                output_folder = os.path.join(bd + '_mm_fixed', vid)
#                 os.makedirs(output_folder, exist_ok=True)
                run_motion_mag_one_video(model, checkpoint, 'run', input_folder, output_folder)
            
    
def run_motion_mag_one_video(model, 
                             checkpoint, 
                             phase, 
                             video, 
                             output,
                            amplification_factor = 6,
                            velocity_mag = True,
                            fl = 0.4,
                            fh = 1,
                            fs = 30,
                            n_filter_tap = 2,
                            filter_type = 'differenceOfIIR'):
    
    print('video', video)
    
    
    try:
        if phase == 'run':
            model.run(checkpoint,
                      video,
                      'jpg',
                      output,
                      amplification_factor,
                      velocity_mag)

        elif phase == 'run_temporal':
            model.run_temporal(checkpoint,
                           video,
                           'jpg',
                           output,
                           amplification_factor,
                           fl,
                           fh,
                           fs,
                           n_filter_tap,
                           filter_type)
    except Exception as err:
        print("Exception found:", err)
        
def instantiate_model_and_run(video_frames_path, output_path, use_temp_filter=True):
        
    configspec = ConfigObj('../../deep_motion_mag/configs/configspec.conf', raise_errors=True)
    config = ConfigObj('../../deep_motion_mag/configs/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3.conf',
                       configspec=configspec,
                       raise_errors=True,
                       file_error=True)
    
    # Validate to get all the default values.
    config.validate(Validator())
    
    network_type = 'ynet_3frames'
    exp_name = 'motion_mag'
    setproctitle.setproctitle(exp_name)
    tfconfig = tf.ConfigProto(allow_soft_placement=True,
                              log_device_placement=False)
    tfconfig.gpu_options.allow_growth = True
    
    with tf.Session(config=tfconfig) as sess:
        model = MagNet3Frames(sess, exp_name, config['architecture'])
        checkpoint = '../../deep_motion_mag/data/training/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3/checkpoint'

        print('running motion mag for vid:',video_frames_path)
        run_motion_mag_one_video(model, 
                                 checkpoint, 
                                 'run' if not use_temp_filter else 'run_temporal', 
                                 video_frames_path, 
                                 output_path)
    return model, sess
        
exp_name= 'o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3'
config_file = '../deep_motion_mag/configs/'+exp_name+'.conf'
config_spec = '../deep_motion_mag/configs/configspec.conf'
base_dirs = ['../../deepfake_data/fb_dfd_release_0.1_final/method_A_crops']
# run_motion_mag_all_folders(base_dirs, config_spec, config_file)

In [12]:
model = instantiate_model_and_run('../../face_1_df', '../../face_1_df_mm')

running motion mag for vid: ../../face_1_df
video ../../face_1_df







 [*] Reading checkpoint...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../deep_motion_mag/data/training/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3/checkpoint/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3-259002


Applying IIR:   0%|          | 0/299 [00:00<?, ?it/s]

Loaded from ckpt: ../../deep_motion_mag/data/training/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3/checkpoint/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3-259002
[*] Load Success
Iteration number is 259002


Applying IIR: 100%|██████████| 299/299 [00:19<00:00, 15.68it/s]


In [10]:
checkpoint = '../../deep_motion_mag/data/training/o3f_hmhm2_bg_qnoise_mix4_nl_n_t_ds3/checkpoint'
run_motion_mag_one_video(model, 
                         checkpoint, 
                         'run_temporal', 
                         '../../face_13_df', 
                         '../../face_13_df_mm',
                        amplification_factor = 5,
                        velocity_mag = True,
                        fl = 0.4,
                        fh = 1,
                        fs = 30,
                        n_filter_tap = 2,
                        filter_type = 'differenceOfIIR')

video ../../face_13_df
Exception found: 'tuple' object has no attribute 'run_temporal'


# Face extraction

In [11]:
#----------------------------------------------
#--- Author         : Ahmet Ozlu
#--- Mail           : ahmetozlu93@gmail.com
#--- Date           : 21st September 2017
#----------------------------------------------

import face_recognition
import cv2
import os
import sys
import matplotlib.pyplot as plt
from multiprocessing import Pool
import multiprocessing as mp
import functools
import ffmpeg

sys.path.append("../face_recognition_crop/utils/")
# import create_csv


def extract_face_from_vid(video, output_path='', m=0.3, show=False):

    if not os.path.isdir(video):
        # Open the input movie file
        input_movie = cv2.VideoCapture(video)
        length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
    else:
        frame_list = [f for f in os.listdir(video) if f.endswith('.jpg')]

    # Initialize some variables
    frame_number = 0
    current_path = os.getcwd()

    while True:
        # Grab next frame
        if os.path.isdir(video):
            if frame_number>=len(frame_list):
                break
            frame = cv2.imread(os.path.join(video, frame_list[frame_number]))
        else:
            # Grab a single frame of video
            ret, frame = input_movie.read()

            if not ret:
                break
                
        frame_number += 1
        
        if not frame_number % 2: continue
            
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)

        # Label the results
        for (top, right, bottom, left) in face_locations:

            if m < 1:
                margin = int((bottom-top)*m)
            else:
                margin = m
            crop_img = frame[top-margin:bottom+margin, left-margin:right+margin]
            os.makedirs(os.path.join(output_path, os.path.basename(video)), exist_ok=True)
            cv2.imwrite(os.path.join(output_path, os.path.basename(video), '%05d.jpg' % (frame_number-1)), crop_img)

        if show:
            plt.imshow(crop_img[:,:,::-1])
            plt.show()

    # All done!
    if not os.path.isdir(video): input_movie.release()
#     create_csv.CreateCsv(current_path + "/face_database/")


def extract_faces_parallel(video_root, output_path, num_threads=100):
    """Extracts faces from video in a parallel fashion."""

    videos = []
    for r, d, f in os.walk(video_root):
        for file in f:
            if '.mp4' in file:
                videos.append(os.path.join(r, file))
#     print(videos)

#     mp.set_start_method('spawn')
    
    func = functools.partial(extract_face_from_vid, output_path=output_path, m=0.3, show=False)
    pool = Pool(num_threads)
    pool.map(func, videos)
    
    
def extract_faces(video_root, output_path):
    videos = []
    for r, d, f in os.walk(video_root):
        for file in f:
            if '.mp4' in file:
                extract_face_from_vid(os.path.join(r, file), output_path=output_path)
                
extract_faces('../../deepfake_data/fb_dfd_release_0.1_final/method_A', 
                '../../deepfake_data/fb_dfd_release_0.1_final/method_A_crops')

In [12]:
from timeit import default_timer as timer

start = timer()
# extract_face_from_vid('../../deepfake_data/fb_dfd_release_0.1_final/method_A/1681757/1681757_D/2076328_1681757_D_001.mp4',
#                      output_path='../../deepfake_data/fb_dfd_release_0.1_final/method_A_crops',show=True)

extract_face_from_vid('../../deepfake_data/fb_dfd_release_0.1_final/method_A_frames/1974002_1061402_B_003.mp4',
                     output_path='../../deepfake_data/fb_dfd_release_0.1_final/method_A_crops',show=False)

end = timer()
print(end-start)

0.325178574770689


In [13]:
from timeit import default_timer as timer

start = timer()
extract_face_from_vid('../../deepfake_data/fb_dfd_release_0.1_final/method_A/1061402/1061402_B/1974002_1061402_B_003.mp4',
                     output_path='../../deepfake_data/fb_dfd_release_0.1_final/method_A_crops',show=False)
end = timer()
print(end-start)

0.00046581123024225235


In [14]:
import dlib.cuda as cuda
print(cuda.get_num_devices())
print(dlib.DLIB_USE_CUDA)

1


NameError: name 'dlib' is not defined